<a href="https://colab.research.google.com/github/Meghraj-Webllisto/language-translation/blob/master/05_02_machine_trainlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
 cd /content/drive/'My Drive'/LanguageTranslation

/content/drive/My Drive/LanguageTranslation


In [32]:
pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 41kB/s 


In [0]:
  %reload_ext autoreload

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17985607393714987162
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8692809632043652550
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8825483963939909903
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4317869455154574364
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [5]:
data_path ="ch-eng.txt"

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
chinese_sentences = []
english_sentences = []

for line in lines:
    try :
        ch, eng =line.split('\t')
        english_sentences.append(eng)
        chinese_sentences.append(ch)
    except:
        continue

print('Dataset Loaded')

Dataset Loaded


In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
chinese_words_counter = collections.Counter([word for sentence in chinese_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} chinese words.'.format(len([word for sentence in chinese_sentences for word in sentence.split()])))
print('{} unique chinese words.'.format(len(chinese_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*chinese_words_counter.most_common(10)))[0]) + '"')


574948 English words.
42213 unique English words.
10 Most common words in the English dataset:
"the" "to" "a" "I" "of" "The" "He" "is" "in" "you"

76026 chinese words.
74356 unique chinese words.
10 Most common words in the French dataset:
"你在看什么？" "坚持。" "。" "天快要下雨了。" "我不怕死。" "她只是個孩子。" "什么时候开始？" "你和谁说话呢？" "能够出一份力太好了。" "Tom"


# Tokenization 

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


# Padding (IMPLEMENTATION)

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]



# Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_chinese_sentences, english_tokenizer, chinese_tokenizer =\
    preprocess(english_sentences, chinese_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_chinese_sequence_length = preproc_chinese_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
chinese_vocab_size = len(chinese_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Chinese sentence length:", max_chinese_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Chinese vocabulary size:", chinese_vocab_size)

Data Preprocessed
Max English sentence length: 32
Max Chinese sentence length: 20
English vocabulary size: 23218
Chinese vocabulary size: 86475


# Model IMPLEMENTATION

In [10]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')







Final Model Loaded


/usr/local/lib/python3.6/dist-packages/keras/engine/sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


# Predicition IMPLEMENTATION

In [0]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_chinese_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(chinese_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_chinese_sentences, batch_size = 128, epochs = 17, validation_split = 0.2)
 
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_chinese_sentences, english_tokenizer, chinese_tokenizer)

Train on 60649 samples, validate on 15163 samples
Epoch 1/17
60649/60649 [==============================] - 247s 4ms/step - loss: 1.0411 - acc: 0.9386 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 2/17
60649/60649 [==============================] - 244s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 3/17
60649/60649 [==============================] - 242s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 4/17
60649/60649 [==============================] - 243s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 5/17
60649/60649 [==============================] - 243s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 6/17
60649/60649 [==============================] - 243s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - val_acc: 0.9429
Epoch 7/17
60649/60649 [==============================] - 244s 4ms/step - loss: 0.9664 - acc: 0.9406 - val_loss: 0.9284 - 

In [0]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb